In [35]:
# autoreload
%load_ext autoreload
%autoreload 2

from dataclasses import replace

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

from Python.pricingmdprunner.simhistories import vec_from_str, prepare_simhistory, allocation_from_simhistory, \
    sold_unsold_allocation_from_simhistory, plot_simhistory_sold_unsold, add_revenue_boxes, add_product_boxes, TEST
from Python.pricingmdprunner.utils import load_yaml

FIGROOT = Path(r"C:\Users\mrkos\scth\projects\MDP_pricing_paper\fig\new_experiments")
EXPERIMENTS_PATH = Path(r"C:\Users\mrkos\scth\projects\MDPPricing\data\ev_variable_resources_higher_demand")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
simh_dfs = []
cfg_type = "_simhistories"
for result_path in EXPERIMENTS_PATH.rglob(f"**/config*{cfg_type}.csv"):
    # print(result_path)
    solver_config_path = result_path.parent/(result_path.stem[:-len(cfg_type)]+".yaml")
    solver_config = load_yaml(solver_config_path)
    # print(solver_config)
    problem_config_path = result_path.parent.parent.parent/"pp_config.yaml"
    problem_config = load_yaml(problem_config_path)
    
    simhistories = pd.read_csv(result_path, dtype={"product":str})
        
    pp_row = pd.DataFrame(problem_config["pp_params"], index=[0,])
    for col in pp_row.columns:
        simhistories[col] = pp_row.loc[0, col]
        
    solver_cfg_row = pd.json_normalize(solver_config)
    for col in solver_cfg_row.columns:
        simhistories[col] = solver_cfg_row.loc[0, col]
        
    simh_dfs.append(simhistories)
    
    # if solver_config["runner"] == "mcts":
    #     break
df = pd.concat(simh_dfs)

In [38]:
traces_dfs = []
for traces_path in EXPERIMENTS_PATH.rglob(f"**/traces*.csv"):
    problem_config_path = traces_path.parent.parent/"pp_config.yaml"
    problem_config = load_yaml(problem_config_path)
    
    traces = pd.read_csv(traces_path, dtype={"product":str})
        
    pp_row = pd.DataFrame(problem_config["pp_params"], index=[0,])
    for col in pp_row.columns:
        traces[col] = pp_row.loc[0, col]
        
    traces_dfs.append(traces)
    
tdf = pd.concat(traces_dfs)

In [39]:
def fix_cols(df):
    if "runner" in df.columns:
        all_replacements = {"mcts": "MCTS", "vi": "VI", "hindsight": "Oracle", "flatrate": "Flatrate"}
        # Do this to prevent the categorical variable from having categoreis not present in dataset
        replacements= {k: all_replacements[k] for k in df.runner.unique()}
        
        df.replace({"runner": replacements }, inplace=True)
        df["runner"] = pd.Categorical(df.runner, categories=list(replacements.values()), ordered=True)
    df.rename(columns={"res_budget_Î¼": "res_budget_mean", "náµ£": "n_r"}, inplace=True)
    
fix_cols(df)
fix_cols(tdf)

## Column meanings
- `r` - revenue
- `nᵣ` - number of requests
- `n_r` - number of resources (timeslots)
- `nₛ` - number of "sold" requests
- `u` - utilization in number of resources (timeslots)
- `res_budget_mean` - mean resource budget
- time - runtime of the experiment in seconds

## Load traces

In [40]:
tdf["prod_vec"] = tdf["product"].apply(vec_from_str)
df["prod_vec"] = df["product"].apply(vec_from_str)

In [41]:
sh = prepare_simhistory(df, tdf, 7, "Oracle", 72)

In [42]:
alloc_sold, alloc_sold_info, sh_sold = allocation_from_simhistory(sh[sh.sold])
alloc_unsold, alloc_unsold_info, sh_unsold = allocation_from_simhistory(sh[~sh.sold])

alloc = np.concatenate([alloc_unsold, alloc_sold], axis=0)
alloc_info = np.concatenate([alloc_unsold_info, alloc_sold_info], axis=0)
sh = pd.concat([sh_sold, sh_unsold])

In [43]:
sold_unsold_allocation_from_simhistory(sh)

,index,trace_id,timestep,budget,action,sold,product,T,c,res_budget_mean,...,solver_params.depth,solver_params.max_iterations,solver_params.belres,solver_params.verbose,prod_vec,allocation_row,n_product_res,revenue,prod_tslot_start,prod_tslot_end
1,478,7,22,1.742979,1.640776e+00,True,000000001000000000000000,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.0,1,1.640776,8,8
3,480,7,28,2.476104,1.192660e+00,True,000000000000011000000000,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",0.0,2,2.385320,13,14
4,481,7,32,2.825945,1.337245e+00,True,000000011000000000000000,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",1.0,2,2.674490,7,8
14,491,7,105,1.564840,1.518217e+00,True,000000000000000100000000,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,1,1.518217,15,15
16,493,7,123,4.838348,1.192660e+00,True,000000000111100000000000,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",0.0,4,4.770641,9,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,544,7,455,1.222413,1.797693e+308,False,000000000000000000010000,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5.0,1,0.000000,19,19
68,545,7,461,2.734961,1.797693e+308,False,000000000000000000011100,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.0,3,0.000000,19,21
69,546,7,463,1.201087,1.797693e+308,False,000000000000000000010000,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8.0,1,0.000000,19,19
75,552,7,561,0.713370,1.797693e+308,False,000000000000000000000001,576,3,1.0,...,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,1,0.000000,23,23


In [44]:
sh = prepare_simhistory(df, tdf, 7, "MCTS", 72)
alloc = sold_unsold_allocation_from_simhistory(sh)

fig = plot_simhistory_sold_unsold(alloc)
fig = add_revenue_boxes(fig, alloc)
fig.show()

In [45]:
sh = prepare_simhistory(df, tdf, 7, "Flatrate", 72)
alloc = sold_unsold_allocation_from_simhistory(sh)

fig = plot_simhistory_sold_unsold(alloc)
fig = add_revenue_boxes(fig, alloc)
fig.show()

# Preparing the plot

In [72]:
sh = prepare_simhistory(df, tdf, 7, "Flatrate", 72)

79

In [46]:
textwidth = 514 # https://tex.stackexchange.com/a/678738/45744
img_width = textwidth/2
img_height = (img_width/6)*4 # 6:4 aspect ratio

In [67]:
sh = prepare_simhistory(df, tdf, 7, "Flatrate", 72)
_, _, alloc = allocation_from_simhistory(sh)

fig = go.Figure()
# Set axes properties
# fig.update_xaxes(range=[0, 24],
#                  showgrid=False
#                  )
y_range = int(alloc.allocation_row.max()) + 1
fig.update_yaxes(range=[0, y_range])
y_offset = 0
fig = add_product_boxes(fig, alloc, y_offset=y_offset, fillcolor='rgba(99, 110, 250, 0.5)')
fig.update_shapes(dict(xref='x', yref='y'))
fig.update_layout(
    width=img_width,  
    height=img_height,
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent background inside the plot
    paper_bgcolor='rgba(0, 0, 0, 0)',  # Transparent background outside the plot
    margin=dict(l=0, r=0, t=0, b=0),  # Adjust margins to make space for border
    xaxis=dict(showticklabels=True, dtick=2, range=[6,24]),
    yaxis=dict(showticklabels=False),
    # Optional: Add border around the entire plot area
    # shapes=[dict(
    #     type='rect',
    #     x0=0, y0=0, x1=1, y1=1,
    #     xref='paper', yref='paper',
    #     line=dict(color='black', width=1)
    # )]
)
fig.show()
fig.write_image(FIGROOT/"trace_analysis"/"trace_analysis-trace.pdf")

In [66]:
sh = prepare_simhistory(df, tdf, 7, "Flatrate", 72)
alloc = sold_unsold_allocation_from_simhistory(sh)
fig = plot_simhistory_sold_unsold(alloc[alloc.sold])
fig = add_revenue_boxes(fig, alloc)
fig.update_layout(
    width=img_width,  
    height=img_height/3,
    xaxis=dict(showticklabels=True, dtick=2,range=[6,24]),
    yaxis=dict(showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)
)
# save
fig.write_image(FIGROOT/"trace_analysis"/"trace_analysis-sold_flatrate.pdf")
fig.show()

In [64]:
sh = prepare_simhistory(df, tdf, 7, "MCTS", 72)
alloc = sold_unsold_allocation_from_simhistory(sh)
fig = plot_simhistory_sold_unsold(alloc[alloc.sold])
fig = add_revenue_boxes(fig, alloc)
fig.update_layout(
    width=img_width,  
    height=img_height/3,
    xaxis=dict(showticklabels=True, dtick=2, range=[6,24]),
    yaxis=dict(showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)
)
# save
fig.write_image(FIGROOT/"trace_analysis"/"trace_analysis-sold_mcts.pdf")
fig.show()

In [65]:
sh = prepare_simhistory(df, tdf, 7, "Oracle", 72)
alloc = sold_unsold_allocation_from_simhistory(sh)
fig = plot_simhistory_sold_unsold(alloc[alloc.sold])
fig = add_revenue_boxes(fig, alloc)
fig.update_layout(
    width=img_width,  
    height=img_height/3,
    xaxis=dict(showticklabels=True, dtick=2, range=[6,24]),
    yaxis=dict(showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)    
)
# save
fig.write_image(FIGROOT/"trace_analysis"/"trace_analysis-sold_oracle.pdf")
fig.show()

In [54]:
img_height

171.33333333333334

In [68]:
72/24

3.0